In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import TavilySearchResults
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os
# 🔐 Load API keys
load_dotenv(".env")
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Simple QA tool (LLMChain)
qa_prompt = PromptTemplate.from_template("Answer clearly: {question}")
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
qa_tool = Tool(
    name="Simple_QA",
    func=qa_chain.run,
    description="Answer the question without hallucination"
)

# Web search tool (Tavily)
search_tool = Tool(
    name="Web_Search",
    func=TavilySearchResults(max_results=3).run,
    description="Search the internet for current info"
)

tools = [qa_tool, search_tool]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_32752\1337553542.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_32752\1337553542.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_32752\1337553542.py:28: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of th

In [2]:
#1) Zero-Shot ReAct (simple + reliable default)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")

C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_32752\1978347885.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\Pragadeesh K\AppData\Local\Temp\ipykernel_32752\1978347885.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("Summarize LangChain



> Entering new AgentExecutor chain...
I should search the internet for information on LangChain and its creator.
Action: Web_Search
Action Input: "LangChain summary" 
Observation: [{'title': 'Summarization with LangChain. Stuff — Map_reduce', 'url': 'https://medium.com/@abonia/summarization-with-langchain-b3d83c030889', 'content': 'Summarization is a critical aspect of natural language processing (NLP), enabling the condensation of large volumes of text into concise summaries. LangChain, a powerful tool in the NLP domain, offers three distinct summarization techniques: `stuff`, `map_reduce`, and `refine`. Each method has its unique advantages and limitations, making them suitable for different scenarios. This article delves into the details of these techniques, their pros and cons, and the ideal scenarios for their [...] I   \n recently wrapped a tutorial on summarization techniques in LangChain. This article covers the basic usage of document summarization techniques and provides in

In [3]:
#2) Conversational ReAct (with memory; non-chat model baseline)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Simple_QA
Action Input: Summarize LangChain in 2 lines, then tell me who created it.
Observation: LangChain is a blockchain platform that aims to bridge language barriers by providing translation services through smart contracts. It was created by a team of developers led by CEO and founder, Alexey Burdyko.
Thought:Do I need to use a tool? No
AI: LangChain is a blockchain platform that provides translation services through smart contracts. It was created by a team of developers led by CEO and founder, Alexey Burdyko.

> Finished chain.


In [5]:
#3) Chat Conversational ReAct (recommended for GPT-3.5/4 chat)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Plan a 3-step study path for LangChain.")
# result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")




> Entering new AgentExecutor chain...
```json
{
    "action": "Simple_QA",
    "action_input": "Plan a 3-step study path for learning languages using LangChain"
}
```
Observation: Step 1: Start by selecting a language to learn on LangChain. Utilize the platform's resources such as interactive lessons, vocabulary exercises, and grammar explanations to build a strong foundation in the language.

Step 2: Practice speaking and listening skills by engaging in language exchange sessions with native speakers on LangChain. This will help improve pronunciation and comprehension, as well as provide real-life context for the language.

Step 3: Continue to expand vocabulary and improve fluency by participating in group discussions, writing exercises, and cultural activities on LangChain. Regular practice and exposure to the language will help solidify your skills and make you more confident in using the language in various situations.
Thought:```json
{
    "action": "Final Answer",
    "action_i

In [6]:
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel

# ✅ Define the input schema using Pydantic
class TitleInput(BaseModel):
    topic: str
    tone: str = "concise"

# ✅ Define the tool function
def title_tool_fn(topic: str, tone: str = "concise") -> str:
    return f"{tone.title()} Title: {topic} in Practice"

# ✅ Create the structured tool
title_tool = StructuredTool.from_function(
    name="TitleMaker",
    func=title_tool_fn,
    description="Generate a title given a topic and an optional tone.",
    args_schema=TitleInput
)

# ✅ Initialize the agent
agent = initialize_agent(
    tools=[title_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Run the agent
result = agent.run("Make a friendly title about LangGraph tutorials.")
print("✅ Result:", result)




> Entering new AgentExecutor chain...
Thought: I can use the TitleMaker tool to generate a friendly title about LangGraph tutorials.

Action:
```
{
  "action": "TitleMaker",
  "action_input": {"topic": "LangGraph tutorials", "tone": "friendly"}
}
```


Observation: Friendly Title: LangGraph tutorials in Practice
Thought:I can provide the final friendly title to the user's request.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Friendly Title: LangGraph tutorials in Practice"
}
```

> Finished chain.
✅ Result: Friendly Title: LangGraph tutorials in Practice


In [7]:
#5)OpenAI Functions (function-calling schema)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
result = agent.run("Search the web for LangGraph docs and give me 3 bullets.")




> Entering new AgentExecutor chain...

Invoking: `Web_Search` with `LangGraph documentation`


[{'title': 'langchain-ai/langgraph: Build resilient language agents as ...', 'url': 'https://github.com/langchain-ai/langgraph', 'content': 'Repository files navigation\n\n   README\n   Contributing\n   MIT license\n   Security\n\nImage 2: LangGraph Logo\n\nImage 3: VersionImage 4: DownloadsImage 5: Open IssuesImage 6: Docs\n\nTrusted by companies shaping the future of agents – including Klarna, Replit, Elastic, and more – LangGraph is a low-level orchestration framework for building, managing, and deploying long-running, stateful agents.\n\nGet started\n\n -> str:\n    """Get weather for a given city."""\n    return f"It\'s always sunny in {city}!" [...] Acknowledgements\n\n[](\n\nLangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.\n\nAbout\n

In [8]:
#6) OpenAI Multi-Functions (can call multiple tools in one step)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True
)
result = agent.run("Answer briefly from QA; also check the web for any updates.")




> Entering new AgentExecutor chain...

Invoking: `Simple_QA` with `{'tool_input': 'What is the capital of France?'}`


The capital of France is Paris.
Invoking: `Web_Search` with `{'tool_input': 'Capital of France'}`


[{'title': 'Paris, the capital of France, is often referred to as the "City of Light ...', 'url': 'https://www.instagram.com/reel/DEVmGsIMGRM/?hl=en', 'content': 'Paris, the capital of France, is often referred to as the "City of Light" and is renowned for its art, culture, and history. It is home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which houses the famous Mona Lisa. Known for its romantic ambiance, Paris boasts charming streets, world-class cuisine, chic boutiques, and vibrant cafes. The city is also a global hub for fashion and luxury, offering a unique blend of historical charm and modern [...] sophistication.', 'score': 0.99980897}, {'title': 'Paris | Definition, Map, Population, Facts, & History - Britannica', 'u

In [9]:
#7) Self-Ask with Search (clarifies, then searches)

search_tool = Tool(
    name="Intermediate Answer",  # This name is REQUIRED
    func=TavilySearchResults(max_results=1).run,
    description="Use this tool to search for missing facts"
)

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    handle_parsing_errors=True
)
result = agent.run("When was the first LangChain release and who founded it?")




> Entering new AgentExecutor chain...
Yes. 
Follow up: Who founded LangChain?
Intermediate answer: [{'title': 'Founder Story: Harrison Chase of LangChain - Frederick AI', 'url': 'https://www.frederick.ai/blog/harrison-chase-langchain', 'content': "Harrison Chase, the visionary co-founder of LangChain, has redefined how developers harness Language Learning Models (LLMs) through a revolutionary open-source framework. From his beginnings in sports analytics to leading roles in machine learning, Chase's journey is marked by his relentless pursuit of simplifying complex AI workflows. His work at LangChain not only empowers developers to build context-aware applications but also sets new standards for collaboration in the AI ecosystem. [...] LangChain. [...] Chase’s vision materialized in October 2022 when the first version of LangChain was released as an open-source Python library. Initially a lightweight wrapper for prompt templates, it quickly evolved to include advanced functionalities